In [1]:
"""
Скрипт для обработки данных  с портала трудвсем для БЦК
"""

'\nСкрипт для обработки данных  с портала трудвсем для БЦК\n'

In [24]:
import pandas as pd
import numpy as np
import openpyxl
import json
import ast
import re

In [23]:
def extract_data_from_list_cell(cell:str,lst_need_keys:list):
    """
    Функция для извлечения данных из словаря в ячейке датафрейма
    """
    try:
        
        lst_lang = ast.literal_eval(cell) # превращаем в список
        if lst_lang:            
            out_str_lst = [] # создаем список содержащий выходные строки
            for lang_dict in lst_lang:
                lst_lang_str = [] # список для хранения значений извлеченных из словаря
                for idx,key in enumerate(lst_need_keys):
                    lst_lang_str.append(lang_dict.get(key,None))             
                    
                lst_lang_str = [value for value in lst_lang_str if value] # отбрасываем None
                single_lang_str = ','.join(lst_lang_str) # создаем строку для одного языка
                out_str_lst.append(single_lang_str)
            return ';'.join(out_str_lst)
                               
        else:
            return 'Не указано'
        

    except FileNotFoundError:
        return 'Не удалось обработать содержимое ячейки'

    

In [52]:
def clear_tag(cell):
    """
    Функция для очистки текста от тегов HTML
    """
    value = str(cell)
    result = re.sub(r'<.*?>','',value)
    result = re.sub(r'&[a-z]*?;','',result)
    return result

def clear_bonus_tag_br(cell):
    """
    Функция для очистки данных в колонке Дополнительные бонусы
    """

    if cell:
        print(cell)
        print('***')
        value = str(cell)
        result = re.sub(r'<.*?>','.',value)

        return result
    else:
        return None

In [4]:
region = 'Республика Бурятия'

In [5]:
# колонки которые нужно оставить и переименовать
dct_name_columns = {'busy_type':'Тип занятости','contact_person':'Контактное лицо','date_create':'Дата размещения вакансии',
                    'date_modify':'Дата изменения вакансии','education':'Требуемое образование','is_quoted':'Квотируемое место',
                   'other_vacancy_benefit':'Дополнительные бонусы','position_requirements':'Требования',
                    'position_responsibilities':'Обязанности','regionName':'Регион','required_experience':'Требуемый опыт работы в годах',
                    'retraining_capability':'Возможность переподготовки','required_certificates':'Требуемые доп. документы',
                   'required_drive_license':'Требуемые водительские права','retraining_grant':'Стипендия',
                    'retraining_grant_value':'Размер стипендии','salary':'Зарплата','salary_min':'Минимальная зарплата','salary_max':'Максимальная зарплата',
                    'schedule_type':'График работы','social_protected_ids':'Социально защищенная категория',
                   'source_type':'Источник вакансии','status':'Статус проверки вакансии','transport_compensation':'Компенсация транспорт',
                    'vacancy_address_additional_info':'Доп информация по адресу вакансии','vacancy_address':'Адрес вакансии',
                    'vacancy_address_latitude':'Долгота адрес вакансии','vacancy_address_longitude':'Широта адрес вакансии',
                   'vacancy_benefit_ids':'Бонусы','vacancy_name':'Вакансия','work_places':'Количество рабочих мест','professionalSphereName':'Сфера деятельности',
                    'full_company_name':'Полное название работодателя','company_inn':'ИНН работодателя','company':'Данные компании',
                    'languageKnowledge':'Данные по языкам','hardSkills':'Данные по хардскиллам','softSkills':'Данные по софтскиллам'}

In [6]:
df = pd.read_csv('data/vacancy.csv',encoding='UTF-8',sep='|',dtype=str)


In [7]:
company_df = pd.read_excel('data/company.xlsx',dtype=str)

In [8]:
company_df

,Краткое название организации,ИНН
0,УУАЗ,0323018510
1,Приборка,0323053578
2,Ника,7536165688
3,РХИ,7802772445
4,Морозовская птицефабрика,5528212010
5,Краевой центр,2466004413


In [9]:
df.shape

(92697, 78)

In [10]:
#df = df[df['regionName'] == region]

In [11]:
df.shape

(92697, 78)

In [12]:
dct_df = dict() # словарь для хранения данных для каждого инн
used_name_sheet = set() # множество для хранения использованных названий листов

In [53]:
# Собираем датафреймы по ИНН
for idx,row in enumerate(company_df.itertuples()):
    name_company = row[1] # название компании
    inn_company = row[2] # инн компании
    temp_df = df[df['company_inn'] ==inn_company] # фильтруем по инн
    temp_df = temp_df[dct_name_columns.keys()]
    temp_df.rename(columns=dct_name_columns,inplace=True)
    # Обрабатываем обычные колонки
    temp_df['Квотируемое место'] = temp_df['Квотируемое место'].apply(lambda x: 'Квотируемое место' if x == 'true' else None)
    temp_df['Дополнительные бонусы'] = temp_df['Дополнительные бонусы'].apply(clear_bonus_tag_br)
    temp_df['Требования'] = temp_df['Требования'].apply(clear_tag)
    
    temp_df['Требуемый опыт работы в годах'] =temp_df['Требуемый опыт работы в годах'].astype(int,errors='ignore')
    temp_df['Минимальная зарплата'] =temp_df['Минимальная зарплата'].astype(int,errors='ignore')
    temp_df['Максимальная зарплата'] =temp_df['Максимальная зарплата'].astype(int,errors='ignore')
    temp_df['Количество рабочих мест'] =temp_df['Количество рабочих мест'].astype(int,errors='ignore')
    # Начинаем извлекать данные из сложных колонок с json
    # данные по работодателю
    temp_df['КПП работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('kpp','Не указано'))
    temp_df['ОГРН работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('ogrn','Не указано'))
    temp_df['Контактный телефон работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('phone','Не указано'))
    temp_df['Email работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('email','Не указано'))
    temp_df['Профиль работодателя'] = temp_df['Данные компании'].apply(lambda x: json.loads(x).get('url','Не указано'))
    
    # Обрабатываем колонку с языками
    temp_df['Знание языков'] = temp_df['Данные по языкам'].apply(lambda x:extract_data_from_list_cell(x,['code_language','level']))
    temp_df['Хардскиллы'] = temp_df['Данные по хардскиллам'].apply(lambda x:extract_data_from_list_cell(x,['hard_skill_name']))
    temp_df['Софтскиллы'] = temp_df['Данные по софтскиллам'].apply(lambda x:extract_data_from_list_cell(x,['soft_skill_name']))
    
    

    
    
    
    
    
    
    
    temp_df.to_excel(f'data/{name_company}.xlsx',index=False)
    

Социальный пакет<br/>Нормальная продолжительность рабочего времени.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Полный рабочий день.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Характер работы постоянная/временная.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Заработная плата( от 37 тыс. руб.) заработная плата указана с учетом удержания НДФЛ.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени.
***
nan
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Постоянная/временная.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Полный рабочий день.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени.
***
Социальный пакет<br/>Нормальная продолжительность рабочего времени. Полный рабочий день  Тип занятости: Полная занятость.
***
Социальный пакет
***
Социальный

***
Предоставление спец.одежды<br/>Столовая
***
Дополнительное медицинское страхование<br/>Предоставление спец.одежды<br/>Столовая
***
Социальный пакет
***
Предоставление спец.одежды<br/>Дополнительное медицинское страхование<br/>Наличие общежития<br/>Столовая
***
Наличие общежития<br/>Дополнительное медицинское страхование<br/>Социальный пакет<br/>Столовая
***
Дополнительное медицинское страхование<br/>Социальный пакет<br/>Столовая<br/>Наличие общежития
***
Дополнительное медицинское страхование<br/>Социальный пакет<br/>Столовая<br/>Наличие общежития
***
nan
***
Социальный пакет
***
nan
***
nan
***
nan
***
<p><strong>Наши преимущества:</strong></p> <ul> <li>доставка транспортом предприятия до работы из районов города (Левый берег, ж/д Вокзал, Нефтяники, Амурский поселок),</li> <li>компенсация питания в уютной столовой на территории предприятия (175 руб. в день). Кормим вкусно и сытно!),</li> <li>бесплатная выдача продукции предприятия (мясо индейки на 2750 руб, примерно 5 кг, 1 раз в 

In [15]:
print('Lindy Booth!')

Lindy Booth!
